In [67]:
import requests
import json

def upload_file(file_path, url='http://127.0.0.1:8000/api/files'):
    """
    Upload a file to the specified URL endpoint.

    Args:
        file_path (str): Path to the file to be uploaded
        url (str): URL endpoint for file upload (default: http://127.0.0.1:8000/api/files)

    Returns:
        requests.Response: Response from the server
    """
    with open(file_path, 'rb') as file:
        files = {'file': file}
        response = requests.post(url, files=files)
    return response

# Example usage:
# response = upload_file('./training_data_sft.jsonl')
response = upload_file('tests/data/training_data_sft.jsonl')
print(response.status_code)  # Print the HTTP status code
response_body = json.loads(response.text)
uploaded_file = response_body["id"]
print(uploaded_file)


200
1a12e2e3-6d9e-40a8-be50-bd05515e765f


In [72]:
#"HuggingFaceTB/SmolLM2-135M-Instruct"
#"Qwen/Qwen2-0.5B-Instruct"
def start_fine_tune(training_file, model_name="HuggingFaceTB/SmolLM2-135M-Instruct", url='http://127.0.0.1:8000/api/fine-tune'):
    """
    Start a fine-tuning job with the specified training file and model.

    Args:
        training_file (str): ID of the uploaded training file
        model_name (str): Name of the base model to fine-tune (default: Qwen/Qwen2-0.5B-Instruct)
        url (str): URL endpoint for fine-tuning (default: http://127.0.0.1:8000/api/fine-tune)

    Returns:
        requests.Response: Response from the server
    """
    headers = {'Content-Type': 'application/json'}
    data = {
        'model': model_name,
        'training_file': training_file
    }
    response = requests.post(url, headers=headers, json=data)
    return response

# Example usage (continuing from previous upload):
fine_tune_response = start_fine_tune(uploaded_file)
print(fine_tune_response.status_code)
print(fine_tune_response.text)

200
{"id":"24649b59-7a45-4bb8-88b7-2bd676a144f1","status":"queued","details":"","fine_tuned_model":null}


In [74]:
def check_job_status(job_id, url_base='http://127.0.0.1:8000/api/job'):
    """
    Check the status of a fine-tuning job.

    Args:
        job_id (str): ID of the fine-tuning job
        url_base (str): Base URL for job status endpoint (default: http://127.0.0.1:8000/api/job)

    Returns:
        requests.Response: Response from the server
    """
    url = f"{url_base}/{job_id}"
    response = requests.get(url)
    return response

# Example usage (continuing from previous fine-tune):
job_response = check_job_status(fine_tune_response.json()["id"])
print(job_response.status_code)
print(job_response.text)

200
{"id":"24649b59-7a45-4bb8-88b7-2bd676a144f1","status":"succeeded","details":"","fine_tuned_model":"./storage/models/ft:smollm2-135m-instruct:1WqC10:20250309_223050"}


In [58]:
def load_model(model_path, device=None):
    """
    Load a fine-tuned model from HuggingFace.

    Args:
        job_response: Response from check_job_status() containing model path
        device: Device to load model on (default: None, will use CUDA if available)

    Returns:
        tuple: (model, tokenizer) loaded from the fine-tuned path
    """
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer

    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # if not model_path:
        # raise ValueError("No fine-tuned model path found in job response")

    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map=device,
        torch_dtype=torch.float16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    return model, tokenizer

# Example usage (continuing from previous job check):
model, tokenizer = load_model(job_response.json()['fine_tuned_model'])


In [59]:
model, tokenizer = load_model('')

OSError: Incorrect path_or_model_id: ''. Please provide either the path to a local folder or the repo_id of a model on the Hub.